In [1]:
import os

import pandas as pd

import cv2 as cv
import numpy as np

from PIL import Image

In [2]:
class LabelNumbers():
    def __init__(self):
        self.data_dir = 'media/test_record_kills_and_players_remaining/'
        self.output_dir = 'media/stable_numbers/'
        
        self.records_dct = {'labeled_numbers' : 'stable_numbers.csv', 'to_be_converted' : f'{self.data_dir}sample_records.csv'}
        
        self.update_df()
        
        self.n_correct_preds = 0
        self.n_incorrect_preds = 0
        
    def ask_human(self, update_df=True):
        if update_df:
            self.update_df()
            
        temp = []
        self.stop_asking_human = False
            
        for _ in range(self.n_existing_loops, self.n_existing_loops + self.n_to_label):
            
            if (_ != 0) and (_ % 100 == 0):
                ask = input('want to keep going? ')
                if (ask != '') and (ask.lower() != 'y') and (ask.lower() != 'yes'):
                    print("ok, let't stop.")
                    break
                else:
                    print("ok, I'll ask again after 100 more screenshots.")
            
            i = self.df_for_conversion.iloc[_]
            
            if _ < 10:
                loop = f'loop_000000{_}'
            elif 10 <= _ < 10**2:
                loop = f'loop_00000{_}'
            elif 10**2 <= row < 10**3:
                loop = f'loop_0000{_}'
            elif 10**3 <= _ < 10**4:
                loop = f'loop_000{_}'
            elif 10**4 <= _ < 10**5:
                loop = f'loop_00{_}'
            elif 10**5 <= _ < 10**6:
                loop = f'loop_0{_}'
            else:
                loop = f'loop_{_}'
            
            new_og_screenshot_file_path = f'{self.output_dir}og_screenshots/{loop}.jpg'
            Image.open(i['og_screenshot']).save(new_og_screenshot_file_path)
            os.remove(i['og_screenshot'])
            
            for rf in i[['k_reference_file', 'pr_reference_file']].values:
                im = cv.imread(rf, cv.IMREAD_UNCHANGED)

                cv.imshow('digits_small', im)
                
                im = Image.fromarray(im)
                im_big = im.resize((im.size[0]*9, im.size[1]*9))

                cv.imshow('digits_large', np.array(im_big))       
                key = cv.waitKey(100)

                if '/n_kills/' in rf:
                    digits = i['n_kills']
                    crop_type = 'k'
                elif '/n_players_remaining/' in rf:
                    digits = i['n_kills']
                    crop_type = 'pr'
                                
                q = input(f'The digits in this image are {digits}? ')
                if q == 'STOP':
                    self.stop_asking_human = True
                    break
                elif (q != '') and (q.lower() != 'y') and (q.lower() != 'yes'):
                    self.n_incorrect_preds += 1
                    try:
                        digits = int(q)
                    except:
                        if (q == 'n') or (q.lower() == 'e') or (q.lower() == 'b'):
                            digits = q
                        else:
                            digits = input('What digits are in this image? ')
                else:
                    self.n_correct_preds += 1
                    
                # convert those that can to numbers from string
                try:
                    digits = float(digits)
                except:
                    pass
                
                new_crop_file_path = f'{self.output_dir}number_crops/{loop}{crop_type}.jpg'
                im.save(new_crop_file_path)
                os.remove(rf)
                
                label_row = [digits, new_crop_file_path, crop_type, i['record_timestamp'], new_og_screenshot_file_path, i['top_left']]
                
                temp.append(label_row)
                
            # update self.records_dct CSVs and self.existing_df 
            # ***LEAVE self.df_for_conversion AS IS***
            if (_ % 21 == 0) and (_ != 0):
                temp_df = self.df_for_conversion.tail(self.n_to_label - _)
                temp_df.to_csv(f'{self.records_dct["to_be_converted"]}', index=False)
                
                temp_df = pd.DataFrame(temp, columns=self.existing_df.columns)
                self.existing_df = pd.concat([self.existing_df, temp_df], axis=0)
                self.existing_df.to_csv(f'{self.records_dct["labeled_numbers"]}', index=False)
                
                temp = []
        
        # update CSVs and DataFrames
        self.df_for_conversion = self.df_for_conversion.tail(self.n_to_label - _)
        self.df_for_conversion.to_csv(f'{self.records_dct["to_be_converted"]}', index=False)

        temp_df = pd.DataFrame(temp, columns=self.existing_df.columns)
        self.existing_df = pd.concat([self.existing_df, temp_df], axis=0)
        self.existing_df.to_csv(f'{self.records_dct["labeled_numbers"]}', index=False)
        
        return self.n_correct_preds, self.n_incorrect_preds
        
    def pull_instance_data(self, reference_file):
        for ref_column in ['k_reference_file', 'pr_reference_file']:
            i_df = self.df_for_conversion.loc[self.df_for_conversion[ref_column] == reference_file]
            if len(i_df) == 1:
                return i_df
            else:
                if len(i_df) != 0:
                    print(f'len(i_df) == {len(i_df)}')
        return None
    
    def update_df(self):
        self.df_for_conversion = pd.read_csv(f'{self.records_dct["to_be_converted"]}')
        self.n_to_label = len(self.df_for_conversion)
        print(f'n to label: {self.n_to_label}')
        
        self.existing_df = pd.read_csv(f'{self.records_dct["labeled_numbers"]}')
        self.n_existing_labeled = len(self.existing_df)
        print(f'n existing: {self.n_existing_labeled}')
        
        if self.n_existing_labeled > 0:
            last_existing_labeled_crop = [f for f in os.listdir(f'{self.output_dir}og_screenshots/') if '.jpg' in f][-1]
            last_existing_loop = last_existing_labeled_crop[-11:-4]
            self.n_existing_loops = int(last_existing_loop)
        else:
            self.n_existing_loops = 0

In [3]:
ln = LabelNumbers()

n to label: 420
n existing: 0


In [4]:
ln.ask_human()

n to label: 420
n existing: 0


The digits in this image are 10?  
The digits in this image are 10?  20
The digits in this image are 19?  10
The digits in this image are 19?  20
The digits in this image are 10?  
The digits in this image are 10?  20
The digits in this image are 13?  10
The digits in this image are 13?  20
The digits in this image are 5?  10
The digits in this image are 5?  20
The digits in this image are 10?  
The digits in this image are 10?  20
The digits in this image are n?  10
The digits in this image are n?  20
The digits in this image are 1?  10
The digits in this image are 1?  20
The digits in this image are 19?  10
The digits in this image are 19?  20
The digits in this image are 10?  
The digits in this image are 10?  20
The digits in this image are 55?  b
The digits in this image are 55?  b
The digits in this image are 12?  10
The digits in this image are 12?  20
The digits in this image are 6?  10
The digits in this image are 6?  20
The digits in this image are 10?  
The digits in this im

ok, let't stop.


(40, 160)